<a href="https://colab.research.google.com/github/gowriks12/chatbot-customer-service/blob/main/Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Customer Service Chatbot**

**Load Large Language Model**

In [1]:
! nvidia-smi

Wed Feb 21 22:31:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install transformers datasets accelerate peft

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.model_selection import train_test_split
from datasets import load_dataset

In [ ]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [13]:
data = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")
df = data["train"].to_pandas()
df = df[["instruction", "response"]]

In [34]:
example_indices = [401, 3000]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT Query:')
    print(df['instruction'][index])
    print(dash_line)
    print('BASELINE HUMAN Response:')
    print(df['response'][index])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT Query:
cance order {{Order Number}}
---------------------------------------------------------------------------------------------------
BASELINE HUMAN Response:
I've decoded that you need assistance with canceling the order with order number {{Order Number}}. We apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:

1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Go to Order History: Once signed in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate Your Order: Look for the order with the order number {{Order Number}} and click on it to view the details.
4. Cancel Your Order: Within the order details page,

In [35]:
for i, index in enumerate(example_indices):
    instruction = df['instruction'][index]
    response = df['response'][index]

    inputs = tokenizer(instruction, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{instruction}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{response}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
cance order {{Order Number}}
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
I've decoded that you need assistance with canceling the order with order number {{Order Number}}. We apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:

1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Go to Order History: Once signed in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate Your Order: Look for the order with the order number {{Order Number}} and click on it to view the details.
4. Cancel Your Order: Within the order details page,

**Zero Shot Prompting**

In [36]:
def zero_shot(query):
  prompt = f"""
  Reply to the query as a customer service agent

  {query}

  Reply:
  """
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)


In [37]:
for i, index in enumerate(example_indices):
  query = df['instruction'][index]
  response = df['response'][index]
  output = zero_shot(query)

  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{query}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{response}')
  print(dash_line)
  print(f'MODEL GENERATION - WITH ZERO SHOT PROMPT ENGINEERING:\n{output}\n')




0
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
cance order {{Order Number}}
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
I've decoded that you need assistance with canceling the order with order number {{Order Number}}. We apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:

1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Go to Order History: Once signed in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate Your Order: Look for the order with the order number {{Order Number}} and click on it to view the details.
4. Cancel Your Order: Within the order details pag

**One Shot Prompting**

In [48]:
def one_shot(query, prompt_indices):
  prompt = ''
  # print(prompt_indices)
  for index in prompt_indices:
    ex_query = df['instruction'][index]
    ex_response = df['response'][index]

    prompt += f"""
    Reply to the query as a customer service agent

    Query:
    {ex_query}

    Response:
    {ex_response}
    """

  prompt += f"""
  Query:
  {query}

  Response:
  """
  # print("One-Shot Prompt\n", prompt)

  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [49]:
for i, index in enumerate(example_indices):
    query = df['instruction'][index]
    response = df['response'][index]
    prompt_indices = [560]
    output = one_shot(query, prompt_indices)

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{query}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{response}')
    print(dash_line)
    print(f'MODEL GENERATION - WITH ONE SHOT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
cance order {{Order Number}}
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
I've decoded that you need assistance with canceling the order with order number {{Order Number}}. We apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:

1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Go to Order History: Once signed in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate Your Order: Look for the order with the order number {{Order Number}} and click on it to view the details.
4. Cancel Your Order: Within the order details page,

**Few Shot Prompting**

In [50]:
def few_shot(query, prompt_indices):
  prompt = ''
  for index in prompt_indices:
    ex_query = df['instruction'][index]
    ex_response = df['response'][index]

    prompt += f"""
    Reply to the query as a customer service agent

    Query:
    {ex_query}

    Response:
    {ex_response}
    """

  prompt += f"""
  Query:
  {query}

  Response:
  """
  # print("Few-Shot Prompt\n", prompt)

  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs, max_new_tokens=20)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [51]:
for i, index in enumerate(example_indices):
    query = df['instruction'][index]
    response = df['response'][index]
    prompt_indices = [560, 2, 8]
    output = few_shot(query, prompt_indices)

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{query}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{response}')
    print(dash_line)
    print(f'MODEL GENERATION - WITH FEW SHOT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
cance order {{Order Number}}
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
I've decoded that you need assistance with canceling the order with order number {{Order Number}}. We apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:

1. Sign into Your Account: Access our platform by signing in to your {{Online Company Portal Info}}.
2. Go to Order History: Once signed in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate Your Order: Look for the order with the order number {{Order Number}} and click on it to view the details.
4. Cancel Your Order: Within the order details page,

**Fine-Tuned Chatbot**

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "/content/drive/MyDrive/Chatbot"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0}).cuda()
model.eval()

In [ ]:
def finetuned_response(query):
  # sample = "instruction: \nI am looking for a product {product ID}, please find it for me \nresponse: "
  input_ids = tokenizer(query, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
  # print(f"{sample}")

  return (tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [ ]:
query = "I am looking for a product {product ID}, please find it for me"
print(finetuned_response(query))